## Task

1. Define a non-homogeneous linear ODE.
2. Solve the homogeneous part first.
3. Apply the method of variation of constants to find the particular solution.
4. Verify the result using `sympy.dsolve`.